[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%205/5.4_sentence_creation.ipynb)

# 4-3. 자연어생성

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## 4-3-1 데이터 불러오기

In [ ]:
# 네이버 영화리뷰 데이터셋 불러오기
file = tf.keras.utils.get_file(
    'ratings_train.txt', 
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', 
    extract=True)

df = pd.read_csv(file, sep='\t')

14630912/14628807 [==============================] - 0s 0us/step


In [ ]:
# 데이터 임의샘플 확인
df[1000:1007]

,id,document,label
1000,9856453,정말 최고의 명작 성인이 되고 본 이집트의 왕자는 또 다른 감동 그자체네요,1
1001,6961803,이영화만 성공 했어도 스퀘어가 에닉스랑 합병 할일은 없었을텐데..,0
1002,8681713,울컥하는 사회현실 ㅠㅠ,1
1003,5348290,기대를하나도안하면 할일없을때보기좋은영화,0
1004,9340549,소림사 관문 통과하기 진짜 어렵다는거 보여준 영화..극장에서 개봉하는거 반갑다..,1
1005,7357684,시리즈안나오나 ㅠㅠㅠㅠㅠㅠㅠㅠ,1
1006,9303587,끝난다는 사실이 너무 슬퍼요. 가슴이 뻥 뚫려버린것같아..,1


## 4-3-2. 데이터 전처리

In [ ]:
# 형태소 분석기 
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 39.8MB/s 
     |████████████████████████████████| 92kB 8.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# 데이터 전처리
def word_tokenization(text):
  return [word for word in okt.morphs(text)]

def preprocessing(df):
  df = df.dropna()
  df = df[1000:2000]  # 샘플 데이터 1000개, 학습시간을 줄이고자 함 
  df['document'] = df['document'].str.replace("[^A-Za-z0-9가-힣ㄱ-ㅎㅏ-ㅣ ]","")
  data =  df['document'].apply((lambda x: word_tokenization(x)))
  return data

In [ ]:
# 텍스트 데이터 1000개 전처리 후 불러오기
review = preprocessing(df)
len(review)

1000

In [ ]:
# 형태소 분리된 데이터 확인
print(review[:10])

1000    [정말, 최고, 의, 명작, 성인, 이, 되고, 본, 이집트, 의, 왕자, 는, 또...
1001    [이영화, 만, 성공, 했어도, 스퀘어, 가, 에, 닉스, 랑, 합병, 할, 일, ...
1002                                 [울컥, 하는, 사회, 현실, ㅠㅠ]
1003       [기대, 를, 하나, 도안, 하, 면, 할, 일, 없을, 때, 보기, 좋은, 영화]
1004    [소림사, 관문, 통과, 하기, 진짜, 어렵다는거, 보여준, 영화, 극장, 에서, ...
1005                              [시리즈, 안, 나오나, ㅠㅠㅠㅠㅠㅠㅠㅠ]
1006        [끝난다는, 사실, 이, 너무, 슬퍼요, 가슴, 이, 뻥, 뚫려, 버린것, 같아]
1007                                             [펑점, 조절]
1008                            [와이, 건, 진짜, 으리, 으리, 한, 데]
1009                                [손발, 이, 오, 그라드, 네, 요]
Name: document, dtype: object


In [ ]:
# 토큰화 및 패딩
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer()

def get_tokens(review):
  tokenizer.fit_on_texts(review)
  total_words = len(tokenizer.word_index)+1
  tokenized_sentences = tokenizer.texts_to_sequences(review)

  input_sequences = []
  for token in tokenized_sentences:
    for t in range(1, len(token)):
        n_gram_sequence = token[:t+1]
        input_sequences.append(n_gram_sequence)

  return input_sequences, total_words

input_sequences, total_words = get_tokens(review)
input_sequences[31:40] # n_gram으로 리스트된 데이터샘플 확인

[[792, 25],
 [792, 25, 539],
 [792, 25, 539, 140],
 [792, 25, 539, 140, 109],
 [338, 9],
 [338, 9, 110],
 [338, 9, 110, 540],
 [338, 9, 110, 540, 90],
 [338, 9, 110, 540, 90, 148]]

In [ ]:
# 단어 사전
print("감동 ==>> ",tokenizer.word_index['감동'])
print("영화 ==>> ",tokenizer.word_index['영화'])
print("코믹 ==>> ",tokenizer.word_index['코믹'])

감동 ==>>  46
영화 ==>>  2
코믹 ==>>  415


In [ ]:
# 문장의 길이 동일하게 맞추기
max_len = max([len(word) for word in input_sequences])
print("max_len:", max_len)
input_sequences = np.array(pad_sequences(input_sequences, 
                                         maxlen=max_len, 
                                         padding='pre'))

max_len: 59


In [ ]:
# 입력텍스트와 타겟
from tensorflow.keras.utils import to_categorical
X = input_sequences[:,:-1]  # 마지막 값은 제외함
y = to_categorical(input_sequences[:,-1], 
                   num_classes=total_words) # 마지막 값만 이진 클래스 벡터로 변환

In [ ]:
# y를 설명하기 위한 예시
a = to_categorical([0, 1, 2, 3], num_classes=4)
a

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

## 4-3-3. 모델

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout

embedding_dim = 256

model = Sequential([
    Embedding(input_dim=total_words, 
              output_dim=embedding_dim, 
              input_length=max_len-1), 
    Bidirectional(LSTM(units=256)),
    Dense(units=total_words, activation='softmax'),
])
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
history = model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
375/375 [==============================] - 57s 61ms/step - loss: 7.8356 - accuracy: 0.0274
Epoch 2/20
375/375 [==============================] - 22s 60ms/step - loss: 7.1486 - accuracy: 0.0297
Epoch 3/20
375/375 [==============================] - 23s 60ms/step - loss: 6.7698 - accuracy: 0.0384
Epoch 4/20
375/375 [==============================] - 22s 60ms/step - loss: 6.1261 - accuracy: 0.0621
Epoch 5/20
375/375 [==============================] - 23s 60ms/step - loss: 5.2681 - accuracy: 0.1059
Epoch 6/20
375/375 [==============================] - 22s 59ms/step - loss: 4.2804 - accuracy: 0.2102
Epoch 7/20
375/375 [==============================] - 22s 59ms/step - loss: 3.2612 - accuracy: 0.3933
Epoch 8/20
375/375 [==============================] - 22s 59ms/step - loss: 2.4284 - accuracy: 0.5584
Epoch 9/20
375/375 [==============================] - 22s 59ms/step - loss: 1.7778 - accuracy: 0.6937
Epoch 10/20
375/375 [==============================] - 22s 59ms/step - loss: 1.277

## 4-3-4. 문장 생성 (추론)

In [ ]:
# 문장생성함수 (시작 텍스트, 생성 단어 개수)
def text_generation(sos, count):
    for _ in range(1, count):
        token_list = tokenizer.texts_to_sequences([sos])[0]
        token_list = pad_sequences([token_list], 
                                   maxlen=max_len-1, 
                                   padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=1) # 최대값 인덱스

        for word, idx in tokenizer.word_index.items():
            if idx == predicted:
                output = word
                break
        sos += " " + output
    return sos

In [ ]:
# argmax 설명: 최대값의 인덱스 반환
data = [[0.1, 0.2, 0.7], [0.3, 0.5, 0.2], [0.4, 0.3, 0.3]]
np.argmax([data], axis=-1)

array([[2, 1, 0]])

In [ ]:
text_generation("연애 하면서", 12)

'연애 하면서 하지 않은 영화 인데 도 귀 자르는 장면 은 기억 에'

In [ ]:
text_generation("꿀잼", 12)

'꿀잼 영화 추억 이다 ㅜㅜ 왜 이 있음 이런 영화 를 보는'

In [ ]:
text_generation("최고의 영화", 12)

'최고의 영화 정말 잘 만들었습니다 이런 영화 자주 나왔으면 좋겠습니다 좋겠습니다 만 제일'

In [ ]:
text_generation("손발 이", 12)

'손발 이 오 그라드 네 요 요 추천 요 로맨스 네 요 본방'